In [1]:
import pandas as pd
import numpy as np

## For plotting
import matplotlib.pyplot as plt
import seaborn as sns

## This sets the plot style
## to have a grid on a dark background
sns.set_style("whitegrid")

In [2]:

from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, precision_score
from sklearn.model_selection import StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

from sklearn.decomposition import PCA

from sklearn.metrics import accuracy_score, confusion_matrix




In [3]:
# Read in your csv file that has the mid features. 

#data = pd.read_csv('../Feature Extraction/midFeaturesTrainSet.csv')
data = pd.read_csv('midFeaturesTrainFinal.csv')


data.head()

print(data.shape)

(4876, 140)


In [4]:
# Split the data into train and test set, stratified by Emotion.


data_train, data_test = train_test_split(data.copy(),
                                   shuffle=True,
                                   random_state=608,
                                   stratify=data.Emotion,
                                   test_size=0.2
                                   )

In [5]:
data.sample(5)

,FileID,actorID,Emotion,SentenceID,zcr_mean,energy_mean,energy_entropy_mean,spectral_centroid_mean,spectral_spread_mean,spectral_entropy_mean,...,delta chroma_4_std,delta chroma_5_std,delta chroma_6_std,delta chroma_7_std,delta chroma_8_std,delta chroma_9_std,delta chroma_10_std,delta chroma_11_std,delta chroma_12_std,delta chroma_std_std
2098,1041_IWW_HAP_XX,1041,HAP,IWW,0.097183,0.025181,2.952492,0.173057,0.191907,0.460020,...,0.022087,0.015760,0.019101,0.022298,0.023059,0.011283,0.008774,0.011074,0.008833,0.008782
1055,1019_WSI_FEA_XX,1019,FEA,WSI,0.086710,0.013681,2.948038,0.162403,0.177849,0.539675,...,0.017477,0.014837,0.015586,0.024109,0.004434,0.039619,0.013754,0.003800,0.005539,0.009264
2546,1050_ITH_ANG_XX,1050,ANG,ITH,0.124939,0.067431,2.903976,0.215148,0.193147,1.175297,...,0.026842,0.016731,0.013254,0.015316,0.022789,0.015017,0.006182,0.012247,0.014102,0.009725
1862,1035_WSI_SAD_XX,1035,SAD,WSI,0.095291,0.012815,2.850804,0.153246,0.168891,0.614320,...,0.020823,0.025505,0.034566,0.012162,0.027521,0.010594,0.013597,0.019151,0.010667,0.010021
3975,1076_IWW_SAD_XX,1076,SAD,IWW,0.140928,0.016603,2.819710,0.227279,0.199751,0.735418,...,0.018149,0.009557,0.012562,0.007259,0.010409,0.006090,0.037344,0.026547,0.010242,0.010197


In [6]:
# Check the percentages of the different emotion categories in the training set

data_train.Emotion.value_counts(normalize=True)

NEU    0.178718
HAP    0.164359
FEA    0.164359
ANG    0.164359
SAD    0.164103
DIS    0.164103
Name: Emotion, dtype: float64

In [7]:
# Check the percentages of the different emotion categories in the test set

data_test.Emotion.value_counts(normalize=True)

NEU    0.179303
DIS    0.164959
ANG    0.163934
SAD    0.163934
FEA    0.163934
HAP    0.163934
Name: Emotion, dtype: float64

In [8]:
y_train = data_train[['Emotion']]
y_test  = data_test[['Emotion']]

X_train = data_train.drop(columns  = ['FileID','actorID', 'Emotion', 'SentenceID'])
X_test  = data_test.drop(columns   = ['FileID','actorID', 'Emotion', 'SentenceID'])

In [9]:
y_test

,Emotion
2589,DIS
628,DIS
3624,ANG
334,ANG
2033,SAD
...,...
4030,ANG
614,SAD
2150,ANG
4016,NEU


In [10]:
# Add a column to the y vectors encoding each of the emotions.

y_train_dummies = pd.get_dummies(y_train)
y_train         = pd.concat([y_train, y_train_dummies], axis=1)

y_test_dummies  = pd.get_dummies(y_test)
y_test          = pd.concat([y_test, y_test_dummies], axis=1)


In [11]:
# Example: try to train a support vector machine with Gaussian radial kernel to distinguish between instances
# where emotion is 'ANG' and instances where emotion is 'HAP'. 

# Get the rows of X_train, X_test corresponding to just the emotions 'ANG' and 'HAP'

X_train_sub = X_train.loc[(y_train.Emotion == 'DIS') | (y_train.Emotion == 'NEU')]
X_test_sub  = X_test.loc[(y_test.Emotion == 'DIS') | (y_test.Emotion == 'NEU')]


# Get the Emotion_ANG column of the ys, with only the rows corresponding to 'ANG' and 'HAP'

y_train_sub = y_train.loc[(y_train.Emotion == 'DIS') | (y_train.Emotion == 'NEU')].Emotion_DIS
y_test_sub  = y_test.loc[(y_test.Emotion == 'DIS') | (y_test.Emotion == 'NEU')].Emotion_DIS



In [12]:
#X_train_sub = X_train
#X_test_sub  = X_test

#y_train_sub = y_train
#y_test_sub  = y_test

In [13]:
print("X_train_sub",  X_train_sub.shape)
print("y_train_sub",  y_train_sub.shape)

X_train_sub (1337, 136)
y_train_sub (1337,)


In [14]:
np.unique(y_test_sub)

array([0, 1], dtype=uint8)

In [15]:
# Build pipeline to first scale the mid feature data, then apply the SVC

pipe = Pipeline([('scale', StandardScaler()),
                 ('svc', SVC(kernel='rbf'))])


# Fit the model to the training data

pipe.fit(X_train_sub, y_train_sub)

    
# Get the model's prediction on the test data

pred = pipe.predict(X_test_sub)



In [16]:
# Build pipeline to first scale the mid feature data, then apply the SVC

pipe = Pipeline([('scale', StandardScaler()),
                 ('svc', SVC(kernel='rbf'))])


# Fit the model t#o the training data

pipe.fit(X_train_sub, y_train_sub)

    
# Get the model's prediction on the test data

pred = pipe.predict(X_test_sub)


In [17]:
# Look at the confusion matrix for the test data :

print("confusion matrix for the test set is:")
print(confusion_matrix(y_test_sub, pred))
print()

# Look at the confusion matrix for the training data:
pred_train = pipe.predict(X_train_sub)
print("confusion matrix for the train set is:")
print(confusion_matrix(y_train_sub, pred_train))


confusion matrix for the test set is:
[[129  46]
 [ 72  89]]

confusion matrix for the train set is:
[[638  59]
 [ 81 559]]
